## Import Dependencies

In [1]:
from dotenv import load_dotenv
load_dotenv()

from tqdm import tqdm
from langchain.chat_models import init_chat_model
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_classic.chains import RetrievalQA

/home/abhigyan/Desktop/projects/rag_langchain_2026api/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# initialize embedding model
embed_model = HuggingFaceEmbeddings()

# initialize chat model
model = init_chat_model(model_provider = 'groq', model="llama-3.1-8b-instant")

Loading weights: 100%|██████████| 199/199 [00:01<00:00, 140.62it/s, Materializing param=pooler.dense.weight]                        
MPNetModel LOAD REPORT from: sentence-transformers/all-mpnet-base-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [3]:
# load the vector store
vector_db_dir = '/home/abhigyan/Desktop/projects/rag_langchain_2026api/vector_db'
collection_name = 'textbook_collections'

vector_store = Chroma(collection_name=collection_name, 
                       embedding_function=embed_model,
                       persist_directory=vector_db_dir)

In [4]:
# use the vector store as retriever in the RetrievalQA chain
retriever = vector_store.as_retriever()

In [5]:
# create QA chain
qa_chain = RetrievalQA.from_chain_type(llm=model, 
                                        retriever=retriever, 
                                        chain_type = 'stuff', 
                                        return_source_documents=True)

In [6]:
query = "What does the document say about Adaptive radiation?"

response = qa_chain.invoke(query)

In [7]:
response

{'query': 'What does the document say about Adaptive radiation?',
 'result': "The document describes Adaptive Radiation as the process of evolution of different species in a given geographical area starting from a point and literally radiating to other areas of geography (habitats). \n\nIt uses the example of Darwin's finches, which were observed by Charles Darwin on the Galapagos Islands. The finches had different beaks, enabling them to eat various food sources, such as seeds or insects. This is an example of how a single species can evolve into different species with different characteristics, adapting to different environments.\n\nThe document also mentions another example of Adaptive Radiation in Australian marsupials, where a number of marsupials, each different from the other, evolved from an ancestral stock within the Australian island continent.",
 'source_documents': [Document(id='4b916cbf-5596-407a-9832-cc2a5de15d28', metadata={'source': '/home/abhigyan/Desktop/projects/rag_

In [8]:
print("Answer: ", response['result'])
print('--'*50)
print("Source Documents: ", [src.metadata for src in response['source_documents']])

Answer:  The document describes Adaptive Radiation as the process of evolution of different species in a given geographical area starting from a point and literally radiating to other areas of geography (habitats). 

It uses the example of Darwin's finches, which were observed by Charles Darwin on the Galapagos Islands. The finches had different beaks, enabling them to eat various food sources, such as seeds or insects. This is an example of how a single species can evolve into different species with different characteristics, adapting to different environments.

The document also mentions another example of Adaptive Radiation in Australian marsupials, where a number of marsupials, each different from the other, evolved from an ancestral stock within the Australian island continent.
----------------------------------------------------------------------------------------------------
Source Documents:  [{'source': '/home/abhigyan/Desktop/projects/rag_langchain_2026api/docs_dir/6. Evoluti